In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

In [ ]:
video_path = '/home/harito/Videos/Cam1.mp4'
# model.track(source=video_path, show=True, persist=True, classes=0, conf=0.35, tracker="bytetrack.yaml")
model.track(source=video_path, show=True, persist=True, classes=0)

In [ ]:
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n-seg.pt')

# Open the video file
video_path = '/home/harito/Videos/Cam1.mp4'
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 230, 0), thickness=10)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO 

def main():
    # Đường dẫn đến ảnh và đường dẫn đến model YOLOv8
    image_path = '/mnt/DataK/Project/SmartCameraSystem/backend/database/data/reid_model/Duc/231231_16h26m53s_screenshot.png'
    yolo_model_path = 'yolov8n-seg.pt'
    frame = cv2.imread(image_path)
    # Load model YOLOv8
    yolo_model = YOLO(yolo_model_path)

    # Run YOLOv8 inference on the frame
    results = yolo_model(frame, classes=0, verbose=False)

    # if not exist person
    if results[0].masks is None:
        return

    # get box object
    box = results[0].boxes[0].xyxy[0]
    box = box.numpy().astype(int)

    # background subtraction
    mask = (results[0].masks.data[0].numpy())

    cv2.imshow('Background remove', mask)

    # Hiển thị ảnh
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [7]:
import cv2
import numpy as np
from ultralytics import YOLO 

def main():
    # Đường dẫn đến ảnh và đường dẫn đến model YOLOv8
    image_path = '/mnt/DataK/Project/SmartCameraSystem/backend/database/data/reid_model/Duc/231231_16h26m53s_screenshot.png'
    yolo_model_path = 'yolov8n-seg.pt'
    frame = cv2.imread(image_path)

    # Load model YOLOv8
    yolo_model = YOLO(yolo_model_path)

    # Run YOLOv8 inference on the frame
    results = yolo_model(frame, classes=0, verbose=False)

    # if not exist person
    if results[0].masks is None:
        return

    # get box object
    x, y, w, h = results[0].boxes.xywh.cpu()[0]
    xmin = int(x - w / 2)
    ymin = int(y - h / 2)
    xmax = int(x + w / 2)
    ymax = int(y + h / 2)
    bounding_box = frame[ymin:ymax, xmin:xmax, :]
    # background subtraction
    mask = (results[0].masks.data[0].numpy())

    # Keep the original image where the mask is 1, set to 0 where the mask is 0
    object_image = bounding_box.copy()
    object_image[mask == 0] = 0

    # Show the result
    cv2.imshow('Object Only', object_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


IndexError: boolean index did not match indexed array along dimension 0; dimension is 390 but corresponding boolean dimension is 640

In [13]:
import cv2
import numpy as np
from ultralytics import YOLO 

def main():
    # Đường dẫn đến ảnh và đường dẫn đến model YOLOv8
    image_path = '/mnt/DataK/Project/SmartCameraSystem/backend/database/data/reid_model/Duc/231231_16h28m59s_screenshot.png'
    yolo_model_path = 'yolov8n-seg.pt'
    frame = cv2.imread(image_path)

    # Load model YOLOv8
    yolo_model = YOLO(yolo_model_path)

    # Run YOLOv8 inference on the frame
    results = yolo_model(frame, classes=0, verbose=False)

    # if not exist person
    if results[0].masks is None:
        return

    # get box object
    box = results[0].boxes[0].xyxy[0]
    box = box.numpy().astype(int)

    # background subtraction
    mask = (results[0].masks.data[0].numpy())

    # Resize mask to match the size of the region of interest (ROI)
    # mask = cv2.resize(mask, (box[2] - box[0], box[3] - box[1]))

    # Keep the original image where the mask is 1, set to 0 where the mask is 0
    object_image = frame.copy()
    object_image[box[1]:box[3], box[0]:box[2]][mask == 0] = 0

    # Show the result
    cv2.imshow('Object Only', object_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    output_path = 'Duc2.png'
    cv2.imwrite(output_path, object_image)


if __name__ == "__main__":
    main()


IndexError: boolean index did not match indexed array along dimension 0; dimension is 654 but corresponding boolean dimension is 640